In [2]:
# Copyright 2021-2022 Boris Shminke
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [1]:
# first we create and reset a OpenAI Gym environment
import sys
if sys.version_info.major == 3 and sys.version_info.minor >= 9:
    from importlib.resources import files
else:
    from importlib_resources import files
import gym
import os

env = gym.make(
    "GymSaturation-v0",
    # for a classical syllogism about Socrates
    problem_list=[
        files("gym_saturation").joinpath(
            os.path.join("resources", "TPTP-mock", "Problems",
                         "TST", "TST003-1.p")
        )
    ],
)
env.reset()
# we can render the environment (that will become the beginning of the proof)
print("starting hypotheses:")
print(env.render("human"))
# our 'age' agent will always select clauses for inference
# in the order they appeared in current proof attempt
action = 0
done = False
while not done:
    observation, reward, done, info = env.step(action)
    action += 1
# SaturationEnv has an additional method
# for extracting only clauses which became parts of the proof
# (some steps were unnecessary to find the proof)
print("refutation proof:")
print(env.tstp_proof)
print(f"number of attempted steps: {action}")

starting hypotheses:
cnf(p_imp_q, hypothesis, ~man(X0) | mortal(X0)).
cnf(p, hypothesis, man(socrates)).
cnf(q, hypothesis, ~mortal(socrates)).
refutation proof:
cnf(x0732affd_d8f4_11ec_8bb2_b9702d53ff10, lemma, mortal(socrates), inference(resolution, [], [p_imp_q, p])).
cnf(x0732b017_d8f4_11ec_8bb2_b9702d53ff10, lemma, $false, inference(resolution, [], [q, x0732affd_d8f4_11ec_8bb2_b9702d53ff10])).
number of attempted steps: 4


In [2]:
# the same example using Vampire backend
env = gym.make(
    "GymVampire-v0",
    problem_list=[
        files("gym_saturation").joinpath(
            os.path.join("resources", "TPTP-mock", "Problems",
                         "TST", "TST003-1.p")
        )
    ],
    vampire_binary_path="vampire"
)
env.reset()
action = 0
done = False
while not done:
    observation, reward, done, info = env.step(action)
    action += 1
print(env.tstp_proof)
print(f"number of attempted steps: {action}")

cnf(1, hypothesis, mortal(X0) | ~man(X0), inference(input, [], [])).
cnf(2, hypothesis, man(socrates), inference(input, [], [])).
cnf(3, hypothesis, ~mortal(socrates), inference(input, [], [])).
cnf(4, hypothesis, mortal(socrates), inference(resolution, [], [2, 1])).
cnf(5, hypothesis, $false, inference(subsumption_resolution, [], [4, 3])).
number of attempted steps: 2
